### Errors

In [1]:
'use strict';

'use strict'

`Error` objects are thrown when runtime errors occur.

In [2]:
try {

    throw new Error('Some descriptive message of the problem, usually freestyle');
    console.log('dead code');

} catch (err) {
    console.log(err);
} finally {
    console.log('Finally, we reached Everest no matter what!');
}

Error: Some descriptive message of the problem, usually freestyle
    at evalmachine.<anonymous>:3:11
    at Script.runInThisContext (node:vm:129:12)
    at Object.runInThisContext (node:vm:305:38)
    at run ([eval]:1020:15)
    at onRunRequest ([eval]:864:18)
    at onMessage ([eval]:828:13)
    at process.emit (node:events:527:28)
    at emit (node:internal/child_process:936:14)
    at process.processTicksAndRejections (node:internal/process/task_queues:83:21)
Finally, we reached Everest no matter what!


One can throw whatever floats its boat: numbers, strings, objects, nuclear waste. Don't ever throw any of those, you tell me why.

In [3]:
try {

    try {

        try {
            throw 10;
        } catch (err) {
            console.log(err);
            throw 'tons';
        }

    } catch (err) {

        console.log(err);
        throw new String('of nuclear waste');
    }

} catch (err) {
    console.log(err);
} finally {
    console.log('Where*tf is the stack trace?');
}

10
tons
[String: 'of nuclear waste']
Where*tf is the stack trace?


Besides the generic `Error` class there are some builtin specific subclasses. Read more on https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Error#error_types

In [4]:
try {

    const someUndefinedVar = undefined;
    someUndefinedVar.toString();

} catch (err) {

    let errClass = err.constructor.name;

    // code below is redundant, just to show the instanceof operator usage
    if (err instanceof TypeError) {
        errClass = TypeError.name;
    } else if (err instanceof Error) {
        errClass = Error.name;
    }

    console.log(`${errClass}: ${err.message}`);
}

TypeError: Cannot read properties of undefined (reading 'toString')


The `Error` class can also be extended for user-defined user exceptions. Remember the first visit to the zoo?

In [5]:
class ZooError extends Error {

    static ERROR_CODE = {
        ABSTRACT_INSTANCE_NOT_ALLOWED: 'ABSTRACT_INSTANCE_NOT_ALLOWED',
        METHOD_NOT_IMPLEMENTED: 'METHOD_NOT_IMPLEMENTED',
        ZOMBIE_NOT_ALLOWED: 'ZOMBIE_NOT_ALLOWED'
    };

    static ERROR_MESSAGE = {
        [ZooError.ERROR_CODE.ABSTRACT_INSTANCE_NOT_ALLOWED]: 'You must extend this class',
        [ZooError.ERROR_CODE.METHOD_NOT_IMPLEMENTED]: 'You must implement this method',
        [ZooError.ERROR_CODE.ZOMBIE_NOT_ALLOWED]: 'This animal is too old to be alive'
    };

    constructor({ code, message = ZooError.ERROR_MESSAGE[code], data }) {
        super(message);

        this.code = code;
        this.data = data;
    }

    toString() {
        // side quest: what does the 2nd and the 3rd parameter of JSON.stringify() mean?
        return JSON.stringify({
            code: this.code,
            message: this.message,
            data: this.data
        }, null, 4) + '\n' + this.stack;
    }
}

In [6]:
try {

    throw new ZooError({
        code: ZooError.ERROR_CODE.METHOD_NOT_IMPLEMENTED,
        data: { method: 'run' }
    });

} catch (err) {
    console.log(err.toString());
}

{
    "code": "METHOD_NOT_IMPLEMENTED",
    "message": "You must implement this method",
    "data": {
        "method": "run"
    }
}
Error: You must implement this method
    at evalmachine.<anonymous>:3:11
    at Script.runInThisContext (node:vm:129:12)
    at Object.runInThisContext (node:vm:305:38)
    at run ([eval]:1020:15)
    at onRunRequest ([eval]:864:18)
    at onMessage ([eval]:828:13)
    at process.emit (node:events:527:28)
    at emit (node:internal/child_process:936:14)
    at process.processTicksAndRejections (node:internal/process/task_queues:83:21)


What about chaining errors? Let's take a very practical example.

In [7]:
class RegistrationError extends Error {

    static ERROR_CODE = {
        REGISTRATION_FAILED: 'REGISTRATION_FAILED'
    };

    static ERROR_MESSAGE = {
        [RegistrationError.ERROR_CODE.REGISTRATION_FAILED]: 'Client registration failed',
    };

    constructor({ code, message = ZooError.ERROR_MESSAGE[code], data, cause }) {
        // in ES2022 the base Error constructor was enhanced with an additional parameter that allows chaining errors
        super(message, { cause });

        this.code = code;
        this.data = data;
    }

    toString() {
        // side quest: what does the 2nd and the 3rd parameter of JSON.stringify() mean?
        return JSON.stringify({
            code: this.code,
            message: this.message,
            data: this.data
        }, null, 4) + '\n' + this.cause;
    }
}

In [8]:
function zooKeeperAdd(animal) {

    const maxAge = 100;

    if (animal.age > maxAge) {

        throw new ZooError({
            code: ZooError.ERROR_CODE.ZOMBIE_NOT_ALLOWED,
            data: {
                species: animal.species,
                age: animal.age,
                maxAge: maxAge
            }
        });
    }
    
    return true;
}

function registerFourLeggedClient(species, age) {
    
    const animal = { species, age };
    try {
        zooKeeperAdd(animal);
    } catch (err) {

        throw new RegistrationError({
            code: RegistrationError.ERROR_CODE.REGISTRATION_FAILED,
            data: { animal },
            cause: err
        });
    }
}

In [9]:
try {
    registerFourLeggedClient('dog', 120);
} catch (err) {
    console.log(err);
}

RegistrationError
    at registerFourLeggedClient (evalmachine.<anonymous>:27:15)
    at evalmachine.<anonymous>:2:5
    ... 6 lines matching cause stack trace ...
    at emit (node:internal/child_process:936:14)
    at process.processTicksAndRejections (node:internal/process/task_queues:83:21) {
  code: 'REGISTRATION_FAILED',
  data: { animal: { species: 'dog', age: 120 } },
  [cause]: ZooError: This animal is too old to be alive
      at zooKeeperAdd (evalmachine.<anonymous>:7:15)
      at registerFourLeggedClient (evalmachine.<anonymous>:24:9)
      at evalmachine.<anonymous>:2:5
      at Script.runInThisContext (node:vm:129:12)
      at Object.runInThisContext (node:vm:305:38)
      at run ([eval]:1020:15)
      at onRunRequest ([eval]:864:18)
      at onMessage ([eval]:828:13)
      at process.emit (node:events:527:28)
      at emit (node:internal/child_process:936:14) {
    code: 'ZOMBIE_NOT_ALLOWED',
    data: { species: 'dog', age: 120, maxAge: 100 }
  }
}
